###Импортим библиотеки

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

###Считываем спарсенные твиты и создаем сырую версию таблицы

In [ ]:
with open('/content/tweets_v2.txt', 'r', encoding='UTF-8') as f:
    data = f.read().split('\n-----\n')
    df = pd.DataFrame (data, columns = ['text_column_name'])
df['rows'] = df['text_column_name'].str.split('\n')
df = df.drop('text_column_name', axis=1)

In [ ]:
df

,rows
0,"[Закрепленный твит, Elon Musk, @elonmusk, ·, 1..."
1,"[Закрепленный твит, Elon Musk, @elonmusk, ·, 1..."
2,"[Закрепленный твит, Elon Musk, @elonmusk, ·, 1..."
3,"[Закрепленный твит, Elon Musk, @elonmusk, ·, 1..."
4,"[Elon Musk ретвитнул(а), SpaceX, @SpaceX, ·, 1..."
...,...
552,"[Elon Musk ретвитнул(а), Tesla, @Tesla, ·, 20 ..."
553,"[Elon Musk, @elonmusk, ·, 20 дек. 2022 г., Oth..."
554,"[Elon Musk ретвитнул(а), Dave Rubin, @RubinRep..."
555,"[Elon Musk ретвитнул(а), Twitter Business, @Tw..."


###Дропаем нерелевантные твиты (последний пустой, дубликаты закрепленного, рекламные и те, которые читает Арбатли)

In [ ]:
useless_twits=[]
for u in range(len(df)):
  if ((df['rows'][u][0]!='Elon Musk ретвитнул(а)') & (df['rows'][u][0]!='Elon Musk')): #посмотрев собранные данные, мы понимаем, что "нужные" твиты начинаются именно так, поэтому ненужные не так :)
    useless_twits.append(u)
useless_twits.remove(0) #нулевой твит у нас закрепленный (имеет другую маску), поэтому оставляем вручную

In [ ]:
df=df.drop(df.index[useless_twits])
df.reset_index(drop=True, inplace=True)

In [ ]:
df

,rows
0,"[Закрепленный твит, Elon Musk, @elonmusk, ·, 1..."
1,"[Elon Musk ретвитнул(а), SpaceX, @SpaceX, ·, 1..."
2,"[Elon Musk ретвитнул(а), SpaceX, @SpaceX, ·, 1..."
3,"[Elon Musk, @elonmusk, ·, 17 ч, Poetry with qu..."
4,"[Elon Musk ретвитнул(а), Déborah, @dvorahfr, ·..."
...,...
540,"[Elon Musk, @elonmusk, ·, 20 дек. 2022 г., As ..."
541,"[Elon Musk ретвитнул(а), Tesla, @Tesla, ·, 20 ..."
542,"[Elon Musk, @elonmusk, ·, 20 дек. 2022 г., Oth..."
543,"[Elon Musk ретвитнул(а), Dave Rubin, @RubinRep..."


###Разделяем данные по столбцам

In [ ]:
df['author'] = df['rows'].apply(lambda x: ''.join(x[0:x.index('·')]))
df['date'] = df['rows'].apply(lambda x: x[(x.index('·')+1)])
df['text'] = df['rows'].apply(lambda x: '/'.join(x[x.index('·')+2:-4]) if x[-1]!='Показать эту ветку' else '/'.join(x[x.index('·')+2:-4]))
df['n_views'] = df['rows'].apply(lambda x: x[-1] if x[-1]!='Показать эту ветку' else x[-2]) #потому что последний элемент может быть "замусорен" "Показать эту ветку"
df['n_likes'] = df['rows'].apply(lambda x: x[-2] if x[-1]!='Показать эту ветку' else x[-3])
df['n_retweets'] = df['rows'].apply(lambda x: x[-3] if x[-1]!='Показать эту ветку' else x[-4])
df['n_replies'] = df['rows'].apply(lambda x: x[-4] if x[-1]!='Показать эту ветку' else x[-5])
df['len_text']=df['text'].apply(lambda x: len(x))
df['is_retweet'] = df['author'].apply(lambda x: int('ретвитнул' in x))

In [ ]:
df

,rows,author,date,text,n_views,n_likes,n_retweets,n_replies,len_text,is_retweet
0,"[Закрепленный твит, Elon Musk, @elonmusk, ·, 1...",Закрепленный твитElon Musk@elonmusk,1 мая,Interview with /@BillMaher,"35,5 млн","205,2 тыс.","33,7 тыс.","24,2 тыс.",26,0
1,"[Elon Musk ретвитнул(а), SpaceX, @SpaceX, ·, 1...",Elon Musk ретвитнул(а)SpaceX@SpaceX,14 ч,Falcon 9’s first stage has landed on the Just ...,"3,1 млн","14,6 тыс.",1 282,430,89,1
2,"[Elon Musk ретвитнул(а), SpaceX, @SpaceX, ·, 1...",Elon Musk ретвитнул(а)SpaceX@SpaceX,14 ч,Liftoff!/0:02 / 0:28,3 млн,"13,5 тыс.",1 809,304,20,1
3,"[Elon Musk, @elonmusk, ·, 17 ч, Poetry with qu...",Elon Musk@elonmusk,17 ч,Poetry with quill /Written on rose /With my ...,"15,7 млн","35,4 тыс.",5 410,5 072,225,0
4,"[Elon Musk ретвитнул(а), Déborah, @dvorahfr, ·...",Elon Musk ретвитнул(а)Déborah@dvorahfr,12 мая,"""The most reliable way to predict the future i...","5,9 млн","20,4 тыс.",2 477,987,80,1
...,...,...,...,...,...,...,...,...,...,...
540,"[Elon Musk, @elonmusk, ·, 20 дек. 2022 г., As ...",Elon Musk@elonmusk,20 дек. 2022 г.,"As (outgoing) Chair of House Intelligence, did...","19,1 млн","202,4 тыс.",46 тыс.,"12,6 тыс.",161,0
541,"[Elon Musk ретвитнул(а), Tesla, @Tesla, ·, 20 ...",Elon Musk ретвитнул(а)Tesla@Tesla,20 дек. 2022 г.,Every Tesla has an electronic powertrain that ...,9 млн,"27,2 тыс.",3 149,1 804,496,1
542,"[Elon Musk, @elonmusk, ·, 20 дек. 2022 г., Oth...",Elon Musk@elonmusk,20 дек. 2022 г.,"Other social media companies too, not just Twi...","20,3 млн","142,4 тыс.","29,8 тыс.",7 203,293,0
543,"[Elon Musk ретвитнул(а), Dave Rubin, @RubinRep...",Elon Musk ретвитнул(а)Dave Rubin@RubinReport,20 дек. 2022 г.,Holy sheep shit. The government was actually p...,"27,8 млн","45,9 тыс.","12,5 тыс.",2 452,501,1


In [ ]:
df.drop('rows', axis=1, inplace=True)

In [ ]:
df #вот такую красивую таблицу получаем

,author,date,text,n_views,n_likes,n_retweets,n_replies,len_text,is_retweet
0,Закрепленный твитElon Musk@elonmusk,1 мая,Interview with /@BillMaher,"35,5 млн","205,2 тыс.","33,7 тыс.","24,2 тыс.",26,0
1,Elon Musk ретвитнул(а)SpaceX@SpaceX,14 ч,Falcon 9’s first stage has landed on the Just ...,"3,1 млн","14,6 тыс.",1 282,430,89,1
2,Elon Musk ретвитнул(а)SpaceX@SpaceX,14 ч,Liftoff!/0:02 / 0:28,3 млн,"13,5 тыс.",1 809,304,20,1
3,Elon Musk@elonmusk,17 ч,Poetry with quill /Written on rose /With my ...,"15,7 млн","35,4 тыс.",5 410,5 072,225,0
4,Elon Musk ретвитнул(а)Déborah@dvorahfr,12 мая,"""The most reliable way to predict the future i...","5,9 млн","20,4 тыс.",2 477,987,80,1
...,...,...,...,...,...,...,...,...,...
540,Elon Musk@elonmusk,20 дек. 2022 г.,"As (outgoing) Chair of House Intelligence, did...","19,1 млн","202,4 тыс.",46 тыс.,"12,6 тыс.",161,0
541,Elon Musk ретвитнул(а)Tesla@Tesla,20 дек. 2022 г.,Every Tesla has an electronic powertrain that ...,9 млн,"27,2 тыс.",3 149,1 804,496,1
542,Elon Musk@elonmusk,20 дек. 2022 г.,"Other social media companies too, not just Twi...","20,3 млн","142,4 тыс.","29,8 тыс.",7 203,293,0
543,Elon Musk ретвитнул(а)Dave Rubin@RubinReport,20 дек. 2022 г.,Holy sheep shit. The government was actually p...,"27,8 млн","45,9 тыс.","12,5 тыс.",2 452,501,1


###Приводим числовые столбцы к адекватному красивому общему виду

In [ ]:
def number_changer(s: str) -> int:
    if 'млн' in s:
      s_new=re.sub("[а-яА-Я]+", "", s)
      s_new=s_new.replace('.','')
      s_new=s_new.replace(',','.')
      num=int(float(s_new)*1000000)
    elif 'тыс' in s:
      s_new=re.sub("[а-яА-Я]+", "", s)
      s_new=s_new.replace('.','')
      s_new=s_new.replace(',','.')
      num=int(float(s_new)*1000)
    elif s=='0:09': #в данных был один очень странный выброс в колонке "n_replies", отображалось "0:09" ответов, что не может быть правдой, поэтому мы просто поставили сюда заглушку правдоподобным числом (не медианой или средним, потому что твитов не очень много и это может исказить картину)
      num=558
    else:
      s = s.replace(' ','')
      num = int(s)
    return num

In [ ]:
df['n_views'] = df['n_views'].apply(lambda x: number_changer(x))
df['n_likes'] = df['n_likes'].apply(lambda x: number_changer(x))
df['n_retweets'] = df['n_retweets'].apply(lambda x: number_changer(x))
df['n_replies'] = df['n_replies'].apply(lambda x: number_changer(x))

In [ ]:
df.head(50) #удостоверяемся, какая красивая табличка получилась :)

,author,date,text,n_views,n_likes,n_retweets,n_replies,len_text,is_retweet
0,Закрепленный твитElon Musk@elonmusk,1 мая,Interview with /@BillMaher,35500000,205200,33700,24200,26,0
1,Elon Musk ретвитнул(а)SpaceX@SpaceX,14 ч,Falcon 9’s first stage has landed on the Just ...,3100000,14600,1282,430,89,1
2,Elon Musk ретвитнул(а)SpaceX@SpaceX,14 ч,Liftoff!/0:02 / 0:28,3000000,13500,1809,304,20,1
3,Elon Musk@elonmusk,17 ч,Poetry with quill /Written on rose /With my ...,15700000,35400,5410,5072,225,0
4,Elon Musk ретвитнул(а)Déborah@dvorahfr,12 мая,"""The most reliable way to predict the future i...",5900000,20400,2477,987,80,1
5,Elon Musk@elonmusk,17 ч,Until the Supreme Court struck down Biden’s va...,40400000,263400,64300,21900,284,0
6,Elon Musk@elonmusk,17 ч,Feels so long ago …/OpIndia.com/@OpIndia_com/·...,25700000,64599,9323,4960,304,0
7,Elon Musk@elonmusk,13 мая,Extremely destructive if accurate,52600000,313500,48500,19100,33,0
8,Elon Musk@elonmusk,13 мая,Explaining Constitutional Amendments,32100000,304800,54600,24900,36,0
9,Elon Musk@elonmusk,13 мая,Raptor V3 just achieved 350 bar chamber pressu...,19100000,92200,8831,6250,208,0


###Конвертим файл для посылки товарищам по команде😁

In [ ]:
df.to_csv('Важный_файл_4.csv')